In [2]:
import pandas as pd
import os
import numpy as np 
from typing import Dict, List, Tuple

In [37]:
def calculate_woe_iv(dataframe : pd.DataFrame, categorical_var:str, target_var:str)-> Tuple[Dict[str,float], Dict[str,float]]:
    """Función que calcula el WOE y IV para una variable categórica respecto a una variable objetivo

    Args:
        dataframe (pd.DataFrame): Dataframe con los datos
        categorical_var (str): Variable sobre la que se quiere calcular el WOE y IV
        target_var (str): Variable objetivo

    Returns:
        Tuple[Dict[str,float], Dict[str,float]]: Diccionarios con el WOE y IV para cada categoría de la variable categórica
    """
    
    # Cálculo de la tabla de frecuencia para la variable categórica respecto a la variable objetivo
    cruce = pd.crosstab(dataframe[categorical_var], dataframe[target_var])
    
    # Cálculo del total de positivos y negativos
    total_posivitos = cruce[1].sum()
    total_negativos = cruce[0].sum()

    # Definición de los diccionarios de WOE y IV
    WOE: Dict[str, float] = {}
    IV : Dict[str, float] = {}
    
    # Cálculo de la proporción de positivos y negativos para cada categoría de la variable
    for categoria in cruce.index:
        eventos_positivos = cruce.loc[categoria, 1]
        eventos_negativos = cruce.loc[categoria, 0]

        # Cálculo del WOE y IV para cada categoría, en el caso de que no haya positivos o negativos se asigna un valor de 0.0001
        ratio_positivos = (eventos_positivos / total_posivitos) if eventos_positivos != 0 else 0.0001
        ratio_negativos = (eventos_negativos / total_negativos) if eventos_negativos != 0 else 0.0001

        # Calculo del WOE y IV
        woe_categoria = np.log(ratio_positivos / ratio_negativos)
        information_value = (ratio_positivos - ratio_negativos) * woe_categoria

        # Se asigna el WOE y IV a los diccionarios
        WOE[categoria] = woe_categoria
        IV[categoria] = information_value
    
    return WOE, IV



In [18]:
data = pd.read_csv(r"C:\Users\andre\OneDrive\Escritorio\Proyecto de Grado\result\db_agrupado\Enero_Agrupado.csv",
                   sep=';')
data = data.drop(columns='DIRECTORIO')

In [19]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6902 entries, 0 to 6901
Data columns (total 43 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   ZONA_TERRITORIAL                      6902 non-null   object 
 1   FACTOR_EXPANSION                      6902 non-null   float64
 2   DPTO                                  6902 non-null   object 
 3   SEXO                                  6902 non-null   object 
 4   EDAD                                  6902 non-null   int64  
 5   PARENTESCO_JEFE_DE_HOGAR              6902 non-null   object 
 6   MADRE_RESIDE_HOGAR                    6902 non-null   object 
 7   PADRE_RESIDE_HOGAR                    6902 non-null   object 
 8   SE_CONSIDERA_CAMPESINO                6902 non-null   object 
 9   COMUNIDAD_ES_CAMPESINA                6902 non-null   object 
 10  ETNIA                                 6902 non-null   object 
 11  ESTADO_CIVIL     

In [26]:
data = data.astype({
    'ZONA_TERRITORIAL':'category',
    'DPTO':'category',
    'SEXO':'category',
    'EDAD':'int16', 
    'PARENTESCO_JEFE_DE_HOGAR':'category', 
    'MADRE_RESIDE_HOGAR':'category',
    'PADRE_RESIDE_HOGAR':'category', 
    'SE_CONSIDERA_CAMPESINO':'category',
    'COMUNIDAD_ES_CAMPESINA': 'category',
    'ETNIA':'category', 
    'ESTADO_CIVIL':'category',
    'AFILIADO_SALUD':'category',
    'LEER_ESCRIBIR':'category', 
    'ACTUALMENTE_ESTUDIA':'category',
    'MAYOR_NIVEL_EDUCATIVO':'category',
    'ORIENTACION_SEXUAL':'category',
    'GENERO':'category', 
    'ACTIVIDAD_OCUPADA_ULTIMA_SEMANA':'int16',
    'TIPO_VIVIENDA':'category', 
    'SERVICIOS_ENERGIA_ELECTRICA':'category',
    'ESTRATO_ENERGIA_ELECTRICA':'category',
    'SERVICIOS_GAS_NATURAL':'category',
    'SERVICIOS_ALCANTARILLADO':'category', 
    'SERVICIOS_RECOLECCION_BASURAS':'category',
    'SERVICIOS_ACUEDUCTO':'category', 
    'NUMERO_HOGARES_VIVIENDA':'int16',
    'NUMERO_CUARTOS_VIVIENDA':'int16',
    'NUMERO_CUARTOS_DORMIR':'int16',
    'TIPO_SANITARIO':'category',
    'COMO_ELIMINA_BASURA':'category',
    'DONDE_OBTIENE_AGUA':'category',
    'DONDE_PREPARA_ALIMENTOS':'category',
    'TIPO_OCUPACION_VIVIENDA':'category',
    'HOGAR_TIENE_CUENTA_CORRIENTE':'category',
    'HOGAR_TIENE_CUENTA_AHORROS':'category', 
    'HOGAR_TIENE_CDT':'category',
    'HOGAR_TIENE_PRESTAMO_COMPRA_VIVIENDA':'category',
    'HOGAR_TIENE_PRESTAMO_COMPRA_VEHICULO':'category',
    'HOGAR_TIENE_PRESTAMO_LIBRE_INVERSION':'category', 
    'HOGAR_TIENE_TARJETA_CREDITO':'category',
    'NUMERO_PERSONAS_HOGAR':'int16', 
    'DISCAPACIDAD':'category'
})

data.ACTIVIDAD_OCUPADA_ULTIMA_SEMANA = data.ACTIVIDAD_OCUPADA_ULTIMA_SEMANA.astype('category')

In [36]:
target = 'ACTIVIDAD_OCUPADA_ULTIMA_SEMANA'
vars_categoric = data.select_dtypes('category').columns.to_list()
vars_categoric.remove('ACTIVIDAD_OCUPADA_ULTIMA_SEMANA')
display(vars_categoric)

['ZONA_TERRITORIAL',
 'DPTO',
 'SEXO',
 'PARENTESCO_JEFE_DE_HOGAR',
 'MADRE_RESIDE_HOGAR',
 'PADRE_RESIDE_HOGAR',
 'SE_CONSIDERA_CAMPESINO',
 'COMUNIDAD_ES_CAMPESINA',
 'ETNIA',
 'ESTADO_CIVIL',
 'AFILIADO_SALUD',
 'LEER_ESCRIBIR',
 'ACTUALMENTE_ESTUDIA',
 'MAYOR_NIVEL_EDUCATIVO',
 'ORIENTACION_SEXUAL',
 'GENERO',
 'TIPO_VIVIENDA',
 'SERVICIOS_ENERGIA_ELECTRICA',
 'ESTRATO_ENERGIA_ELECTRICA',
 'SERVICIOS_GAS_NATURAL',
 'SERVICIOS_ALCANTARILLADO',
 'SERVICIOS_RECOLECCION_BASURAS',
 'SERVICIOS_ACUEDUCTO',
 'TIPO_SANITARIO',
 'COMO_ELIMINA_BASURA',
 'DONDE_OBTIENE_AGUA',
 'DONDE_PREPARA_ALIMENTOS',
 'TIPO_OCUPACION_VIVIENDA',
 'HOGAR_TIENE_CUENTA_CORRIENTE',
 'HOGAR_TIENE_CUENTA_AHORROS',
 'HOGAR_TIENE_CDT',
 'HOGAR_TIENE_PRESTAMO_COMPRA_VIVIENDA',
 'HOGAR_TIENE_PRESTAMO_COMPRA_VEHICULO',
 'HOGAR_TIENE_PRESTAMO_LIBRE_INVERSION',
 'HOGAR_TIENE_TARJETA_CREDITO',
 'DISCAPACIDAD']

In [40]:
## Cálculo del WOE e IV
iv_df = pd.DataFrame(columns=['IV'])

# Calculate WOE and IV for each variable in the categorical dataframe
for column in vars_categoric:
    woe, iv = calculate_woe_iv(data, column, target)
    sum_iv = sum([i for i in iv.values()])
    iv_df.loc[column, 'IV'] = sum_iv

# Display the resulting dataframe
print("Resultados del Cálculo de IV")
display(iv_df.sort_values(by='IV', ascending=False))

print("Variables con IV mayor a 0.02, es decir, predictores aceptables")
display(iv_df[iv_df['IV'] > 0.02].sort_values(by='IV', ascending=False))

iv_columns = list(iv_df[iv_df['IV'] > 0.02].sort_values(by='IV', ascending=False).index)[:4]

Resultados del Cálculo de IV


,IV
PARENTESCO_JEFE_DE_HOGAR,0.281619
MADRE_RESIDE_HOGAR,0.21639
ESTADO_CIVIL,0.192598
DPTO,0.155438
PADRE_RESIDE_HOGAR,0.072577
TIPO_OCUPACION_VIVIENDA,0.064025
MAYOR_NIVEL_EDUCATIVO,0.050143
TIPO_VIVIENDA,0.036521
DONDE_OBTIENE_AGUA,0.027612
ACTUALMENTE_ESTUDIA,0.02448


Variables con IV mayor a 0.02, es decir, predictores aceptables


,IV
PARENTESCO_JEFE_DE_HOGAR,0.281619
MADRE_RESIDE_HOGAR,0.21639
ESTADO_CIVIL,0.192598
DPTO,0.155438
PADRE_RESIDE_HOGAR,0.072577
TIPO_OCUPACION_VIVIENDA,0.064025
MAYOR_NIVEL_EDUCATIVO,0.050143
TIPO_VIVIENDA,0.036521
DONDE_OBTIENE_AGUA,0.027612
ACTUALMENTE_ESTUDIA,0.02448


In [43]:
# Concatenación a lista de variables importantes
demograficas = ['DPTO','SEXO','ETNIA','GENERO','ORIENTACION_SEXUAL','ESTRATO_ENERGIA_ELECTRICA',
                'MAYOR_NIVEL_EDUCATIVO','TIPO_OCUPACION_VIVIENDA']

to_model_categoric = list(set(iv_columns + demograficas))
display(to_model_categoric)


['TIPO_OCUPACION_VIVIENDA',
 'ETNIA',
 'GENERO',
 'ESTADO_CIVIL',
 'ORIENTACION_SEXUAL',
 'ESTRATO_ENERGIA_ELECTRICA',
 'MAYOR_NIVEL_EDUCATIVO',
 'MADRE_RESIDE_HOGAR',
 'PARENTESCO_JEFE_DE_HOGAR',
 'SEXO',
 'DPTO']

In [46]:
# Variables Numéricas
vars_numeric = data.select_dtypes('int16').columns.to_list()

In [49]:
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif

In [59]:
# Prueba t-Student
selector_anova = SelectKBest(f_classif, k=3)
X_ttest = selector_anova.fit_transform(data[vars_numeric], data[target])

for col, value in zip(selector_anova.feature_names_in_, selector_anova.scores_):
    print("Columna: {}, Valor: {}".format(col,value))

Columna: EDAD, Valor: 54.17416763305664
Columna: NUMERO_HOGARES_VIVIENDA, Valor: 2.2768852710723877
Columna: NUMERO_CUARTOS_VIVIENDA, Valor: 40.1308479309082
Columna: NUMERO_CUARTOS_DORMIR, Valor: 56.36643981933594
Columna: NUMERO_PERSONAS_HOGAR, Valor: 47.824851989746094


In [60]:
# Prueba t-Student
selector_mutual = SelectKBest(mutual_info_classif, k=3)
X_mutual = selector_mutual.fit_transform(data[vars_numeric], data[target])

for col, value in zip(selector_mutual.feature_names_in_, selector_mutual.scores_):
    print("Columna: {}, Valor: {}".format(col,value))

Columna: EDAD, Valor: 0.005362702935487906
Columna: NUMERO_HOGARES_VIVIENDA, Valor: 0.006014229542990135
Columna: NUMERO_CUARTOS_VIVIENDA, Valor: 0.008309271085563896
Columna: NUMERO_CUARTOS_DORMIR, Valor: 0.010764084267928409
Columna: NUMERO_PERSONAS_HOGAR, Valor: 0.005140655396872917


In [61]:
selector_mutual.get_feature_names_out()

array(['NUMERO_HOGARES_VIVIENDA', 'NUMERO_CUARTOS_VIVIENDA',
       'NUMERO_CUARTOS_DORMIR'], dtype=object)

In [65]:
data[vars_numeric+[target]].corr()

,EDAD,NUMERO_HOGARES_VIVIENDA,NUMERO_CUARTOS_VIVIENDA,NUMERO_CUARTOS_DORMIR,NUMERO_PERSONAS_HOGAR,ACTIVIDAD_OCUPADA_ULTIMA_SEMANA
EDAD,1.000000,-0.001559,0.002459,-0.061347,-0.091382,0.088229
NUMERO_HOGARES_VIVIENDA,-0.001559,1.000000,-0.078384,-0.083216,-0.032475,0.018170
NUMERO_CUARTOS_VIVIENDA,0.002459,-0.078384,1.000000,0.774530,0.384103,-0.076042
NUMERO_CUARTOS_DORMIR,-0.061347,-0.083216,0.774530,1.000000,0.583132,-0.090014
NUMERO_PERSONAS_HOGAR,-0.091382,-0.032475,0.384103,0.583132,1.000000,-0.082967
ACTIVIDAD_OCUPADA_ULTIMA_SEMANA,0.088229,0.018170,-0.076042,-0.090014,-0.082967,1.000000


In [67]:
to_model_numeric = ['EDAD','NUMERO_CUARTOS_VIVIENDA','NUMERO_CUARTOS_DORMIR','NUMERO_PERSONAS_HOGAR']

In [81]:
final_columns = to_model_numeric + to_model_categoric + ['FACTOR_EXPANSION'] + [target]

In [78]:
result = data[final_columns]

In [80]:
path = r"C:\Users\andre\OneDrive\Escritorio\Proyecto de Grado\result\db_feature_selection"
month = 'Enero'
path_export = os.path.join(path, month +'_Agrupado' +'.csv')
result.to_csv(path_export, index=False, sep=';')